In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

import time
import re

In [22]:
import pandas as pd

In [32]:

def run_driver(url):
    service = Service(executable_path=ChromeDriverManager().install())
    options = Options()
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(6)
    return driver

def check_element_html(parent_tag, xpath, one_tag=True):
    try:
        return parent_tag.find_element(By.XPATH, xpath) if one_tag else parent_tag.find_elements(By.XPATH, xpath)
    except NoSuchElementException:
        return None

def get_specifications(parent_tags_list, data_watches):
    # i = 0
    for parent_tag in parent_tags_list:
        product_id = parent_tag.get_attribute('data-id')
        gender = None
        face_diameter_mm = None
        machine_type = None
        face_thinness_mm = None
        glass_surface_material = None
        
        name_tag = check_element_html(parent_tag, "./a[@class='main-contain ']//h3[@class='fashionWatch-name']")
        if name_tag is not None:
            title = name_tag.text.strip()

            gender = re.findall(r'(?:Nam|Nữ|Cặp đôi|Trẻ em|Unisex)', title)[0]
            face_diameter = re.findall(r'[\d.]+(?=\s?mm)', title)
            if len(face_diameter) > 0:
                face_diameter_mm = float(face_diameter[0])
            
        specification_tags = check_element_html(parent_tag, ".//div[@class='utility']//p", False)
        if specification_tags is not None:
            data = [tag.text for tag in specification_tags]
            machine_type = data[0].split(':')[1].strip()
            
            if len(data) > 2:
                face_thinness_mm = float(data[1].split(':')[1].replace('mm', '').strip())
                glass_surface_material = data[2].split(':')[1].strip()
            else:
                glass_surface_material = data[1].split(':')[1].strip()
                               
        data_watches.append([product_id, gender, face_diameter_mm, machine_type, face_thinness_mm, glass_surface_material])
        # print(f'############ {i} ############')
        # i += 1
  
def watch():
    url = "https://www.thegioididong.com/dong-ho-deo-tay-casio"
    driver = run_driver(url)
    quantity_watches = driver.find_element(By.XPATH, "//section[@id='categoryPage']//div[@class='box-sort ']/p[@class='sort-total']/b").text
    pages = int(quantity_watches) // 20
    
    data_watches = []   
    for page in range(pages):
        new_url = f"https://www.thegioididong.com/dong-ho-deo-tay-casio#c=7264&m=17875&o=14&pi={page}"
        driver = run_driver(new_url)

        parent_tags_list = driver.find_elements(By.XPATH, "//li[@class=' item  __cate_7264']")
        get_specifications(parent_tags_list, data_watches)  
        
    df = pd.DataFrame(data_watches, columns=['product_id', 'gender', 'face_diameter_mm', 'machine_type', 'face_thinness_mm', 'glass_surface_material'])
    return df

watches_df = watch()
watches_df
# watch_specifications()

# Error in page 7, error part: glass_surface_material

IndexError: list index out of range

In [7]:
s = 'Độ mỏng mặt: 9.3 mm'
float(s.split(':')[1].replace(' mm', '').strip())

9.3

In [16]:
271//20

13